In [ ]:
# Import necessary modules and classes
from langchain.schema import Document  # Represents a document with content and metadata
from langchain_openai import OpenAIEmbeddings  # Provides embeddings using OpenAI models
from langchain_community.vectorstores import Chroma  # A vector store for document retrieval

# Initialize the embedding function using OpenAI models
embedding_function = OpenAIEmbeddings()

# Define a list of documents with their content and metadata
docs = [
    Document(
        page_content="Peak Performance Gym was founded in 2015 by former Olympic athlete Marcus Chen. With over 15 years of experience in professional athletics, Marcus established the gym to provide personalized fitness solutions for people of all levels. The gym spans 10,000 square feet and features state-of-the-art equipment.",
        metadata={"source": "about.txt"}
    ),
    Document(
        page_content="Peak Performance Gym is open Monday through Friday from 5:00 AM to 11:00 PM. On weekends, our hours are 7:00 AM to 9:00 PM. We remain closed on major national holidays. Members with Premium access can enter using their key cards 24/7, including holidays.",
        metadata={"source": "hours.txt"}
    ),
    Document(
        page_content="Our membership plans include: Basic (₹1,500/month) with access to gym floor and basic equipment; Standard (₹2,500/month) adds group classes and locker facilities; Premium (₹4,000/month) includes 24/7 access, personal training sessions, and spa facilities. We offer student and senior citizen discounts of 15% on all plans. Corporate partnerships are available for companies with 10+ employees joining.",
        metadata={"source": "membership.txt"}
    ),
    Document(
        page_content="Group fitness classes at Peak Performance Gym include Yoga (beginner, intermediate, advanced), HIIT, Zumba, Spin Cycling, CrossFit, and Pilates. Beginner classes are held every Monday and Wednesday at 6:00 PM. Intermediate and advanced classes are scheduled throughout the week. The full schedule is available on our mobile app or at the reception desk.",
        metadata={"source": "classes.txt"}
    ),
    Document(
        page_content="Personal trainers at Peak Performance Gym are all certified professionals with minimum 5 years of experience. Each new member receives a complimentary fitness assessment and one free session with a trainer. Our head trainer, Neha Kapoor, specializes in rehabilitation fitness and sports-specific training. Personal training sessions can be booked individually (₹800/session) or in packages of 10 (₹7,000) or 20 (₹13,000).",
        metadata={"source": "trainers.txt"}
    ),
    Document(
        page_content="Peak Performance Gym's facilities include a cardio zone with 30+ machines, strength training area, functional fitness space, dedicated yoga studio, spin class room, swimming pool (25m), sauna and steam rooms, juice bar, and locker rooms with shower facilities. Our equipment is replaced or upgraded every 3 years to ensure members have access to the latest fitness technology.",
        metadata={"source": "facilities.txt"}
    )
]

# Create a Chroma vector store from the documents using the embedding function
db = Chroma.from_documents(docs, embedding_function)

In [ ]:
# Initialize a retriever from the vector store
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 3})  # Use Maximal Marginal Relevance (MMR) to retrieve top 3 relevant documents

In [ ]:
# Query the retriever with a question
retriever.invoke("Who founded the gym and what are the timings?")  # Retrieve relevant documents based on the query

In [ ]:
# Import the ChatPromptTemplate for creating prompts
from langchain_core.prompts import ChatPromptTemplate

# Define a template for the question-answering task
template = """ 
Answer the question based only on the following context: {context}
Question: {question}
"""

# Create a prompt template from the defined template
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Import necessary modules for output parsing and LLM interaction
from langchain_core.output_parsers import StrOutputParser  # Parses the output into a string
from langchain_openai import ChatOpenAI  # Provides access to OpenAI's chat models

# Initialize the language model with a specific configuration
llm = ChatOpenAI(model="gpt-4o")

# Define a function to format documents into a single string
def format_docs(docs): 
    return "\n\n".join(doc.page_content for doc in docs)  # Concatenate document content with double newlines

# Create a question-answering chain
qa_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x)),  # Format retrieved documents as context
        "question": lambda x: x  # Pass the input question directly
    }
    | prompt  # Use the defined prompt template
    | llm  # Use the language model for generating answers
    | StrOutputParser()  # Parse the output into a string
)

# Invoke the QA chain with a specific question
qa_chain.invoke("Who is the owner and what are the timings?")  # Generate an answer based on the question and context

'The owner mentioned is Antonio Rossi, who owns Bella Vesta. The timings provided are for Peak Performance Gym, which is open Monday through Friday from 5:00 AM to 11:00 PM and on weekends from 7:00 AM to 9:00 PM. The gym is closed on major national holidays, but members with Premium access can enter 24/7 using their key cards, including holidays.'